In [1]:
from pathlib import Path
import pandas as pd, json, re, numpy as np, matplotlib.pyplot as plt
TABLES = Path("../results/tables")
files = sorted(TABLES.glob("nullcal_*_p*.csv"))
df = pd.concat([pd.read_csv(f).assign(file=f.name) for f in files], ignore_index=True)
df["p"] = df["file"].str.extract(r'_p(\d+)_').astype(int)

In [ ]:
# FDP(null) vs B (grouped by p, α)
import re
fdp_cols = [c for c in df.columns if c.startswith("fdp_lctb_")]
F = df.melt(id_vars=["file","p"], value_vars=fdp_cols, var_name="metric", value_name="fdp")
F["alpha"] = F["metric"].str.extract(r'_(0\.05|0\.10)')
F["B"] = F["metric"].str.extract(r'_B(\d+)').astype(float)
agg = F.groupby(["p","alpha","B"])["fdp"].agg(mean="mean", se=lambda x:x.std(ddof=1)/np.sqrt(len(x))).reset_index()
display(agg.head())

In [ ]:
# plot & annotate chosen B
with open("../results/defaults.json","r") as f:
    defaults = json.load(f)

for pval in sorted(agg["p"].unique()):
    for alpha in ["0.05","0.10"]:
        sub = agg[(agg["p"]==pval)&(agg["alpha"]==alpha)]
        if sub.empty: continue
        plt.figure()
        plt.errorbar(sub["B"], sub["mean"], yerr=sub["se"], fmt="o-")
        chosen = defaults.get(str(pval),{}).get(alpha,{})
        if "B" in chosen:
            plt.axvline(chosen["B"], linestyle="--")
            plt.title(f"FDP(null) vs B (p={pval}, α={alpha}) — chosen B={chosen['B']}")
        else:
            plt.title(f"FDP(null) vs B (p={pval}, α={alpha})")
        plt.xlabel("B")
        plt.ylabel("mean FDP")
        plt.ylim(0, max(0.2, sub["mean"].max()+0.05))
        plt.show()